In [1]:
from io import StringIO
import os
from time import sleep
from threading import Thread
import traceback
import subprocess

from hand import *
from viz.main import pygame_mainloop_thread_start

%load_ext autoreload
%autoreload 2

pygame 2.1.3 (SDL 2.26.4, Python 3.10.9)
Hello from the pygame community. https://www.pygame.org/contribute.html
Assets: /home/diego/ai/hand-transformer/viz/assets
Images: /home/diego/ai/hand-transformer/viz/assets/images


In [2]:
# Start the pygame app in a separate thread
quit_pygame = pygame_mainloop_thread_start(globals())

<function viz.main.pygame_mainloop_thread_start.<locals>.<lambda>()>

In [5]:
from viz.engine import *
from viz.utils import *

In [3]:
quit_pygame()

NameError: name 'quit_pygame' is not defined

In [6]:
prompt = "0010"
task = Task.from_texts([
    "0000",
    "0011",
    "0100",
    "0111",
    "1000",
    "1010",
    "1101",
    "1111",
])

model = torch.load("./models/ITE-25k-epochs")
model = to_new_implem(model)
from solutions.if_then_else import if_then_else
model = if_then_else()

perfs = Perfs([0])

In [7]:
original = if_then_else()
loss_original = task.loss(original)

optimizer = torch.optim.SGD(model.parameters(),
                            lr=0.2,
                            weight_decay=0.01)

In [8]:
def draw(gfx: GFX):
    r = transformer_outline(gfx, model)
    # transformer_outline(gfx, original, r.topright + Vec2(40, 0))
    loss = perfs.loss[-1]
    r = gfx.texts(
        f"Loss trained:  {loss:.8f}",
        f"Loss original: {loss_original:.8f}",
        f"Loss diff: {loss - loss_original:.8f}",
        f"Accuracy trained: {task.test(model):.2%}",
        bottomleft=r.topleft,
        color='white',
    )

    q = model.blocks[0].attention.heads[1].queries
    k = model.blocks[0].attention.heads[1].keys
    emb = model.embedding.weight
    qk = q @ k.T
    r = draw_matrix(gfx, q, topleft=(-250, 50), title="Q")
    draw_matrix(gfx, k, topleft=r.topright + Vec2(20, 0), title="K")
    r = draw_matrix(gfx, emb[0] @ qk, topleft=r.bottomleft, title="QK·E_0")
    r = draw_matrix(gfx, emb[1] @ qk, topleft=r.topright + Vec2(50, 0), title="QK·E_1")
    v = qk @ emb[1]
    emb_v = emb @ v
    r = draw_matrix(gfx, emb_v, topleft=r.bottomleft, title="V")
    pe = model.position_encoder
    r = draw_matrix(gfx, pe @ v, topleft=r.bottomleft + Vec2(0, 50), title="PE")

    gfx.plot(perfs.loss[-10000:], center=(200, -100), title="Loss", color="red")


: 

In [7]:
def logic():
    train(model, task, epochs=1, perfs=perfs, optimizer=optimizer)
    pass

In [16]:
@pyapp.state.do_later(1)
def do_next():
    with torch.no_grad():
        emb = model.embedding.weight
        emb[0].abs_()
        emb[1] = -emb[1].abs()
        # for param in model.parameters():
        #     a = 0.2
        #     param.clamp_(-a, a)

In [23]:
class TimeControl(Object):
    def __init__(self, initial_state):
        self.time = 0
        self.paused = False
        self.states = [initial_state]

        super().__init__((0, 0))

    @property
    def current_state(self):
        return self.states[self.time]

    def create_inputs(self):
        return super().create_inputs().union({
            "pause-time": Button(pygame.K_SPACE)
                .on_press(self.toggle_pause),
            "next-time": Button(pygame.K_RIGHT)
                .on_press_repeated(lambda _: self.move_time(1), 0.1)
                .on_double_press(lambda _: self.move_time(10)),
            "prev-time": Button(pygame.K_LEFT)
                .on_press_repeated(lambda _: self.move_time(-1), 0.1)
                .on_double_press(lambda _: self.move_time(-10)),
        })

    def toggle_pause(self):
        self.paused = not self.paused

    def move_time(self, delta):
        self.time = max(0, min(len(self.states), self.time + delta))

    def draw(self, gfx: GFX):
        rect = pygame.Rect(0, 0, gfx.surf.get_width(), 30)
        gfx.box(rect, color="black", abs_position=True)
        pygame.draw.line(gfx.surf, "white", rect.bottomleft, rect.bottomright)
        r = gfx.text(f"Time: {self.time + 1} / {len(self.states)}", midleft=rect.midleft, color="white", abs_position=True)
        # 400 px progress bar
        r = gfx.rect(r.right + 10, rect.centery -10, 400, 20, color="white", width=1, abs_position=True)

    def append_state(self, state):
        # Advance the time if we were waiting for the next state
        if not self.paused and self.time == len(self.states):
            self.time += 1
        self.states.append(state)

# For reloading
try:
    timer.alive = False
except NameError:
    pass

timer = pyapp.state.add(TimeControl(model))



In [9]:
class TensorView(Object):
    def __init__(self, tensor_or_getter, neuron_size=10, title="", legend=False, **anchor):
        if callable(tensor_or_getter):
            self._tensor_or_getter = tensor_or_getter
        else:
            self._tensor_or_getter = lambda: tensor_or_getter

        t = self.tensor
        self.shape = tuple(t.shape)
        self.title = title
        self.legend = legend
        self.anchor = anchor
        self.neuron_size = neuron_size

        rect = self.geometry(**anchor)

        super().__init__(rect.topleft, rect.size, (0, 0))

    @property
    def tensor(self) -> torch.Tensor:
        return self._tensor_or_getter()

    def geometry(self, ret_heatmap_rect=False, **anchor) -> pygame.Rect:
        assert len(anchor) == 1, "Only one anchor is allowed"
        
        size = self.neuron_size
        
        heatmap_rect = Rect(0, 0, self.shape[1] * size, self.shape[0] * size)
        full_rect = heatmap_rect.copy()
        if self.title:
            title_height = font().size(self.title)[1]
            full_rect.height += title_height
            heatmap_rect.y += title_height
        if self.legend:
            full_rect.width += size
            heatmap_rect.x += size
        anchor, anchor_pos = anchor.popitem()
        setattr(full_rect, anchor, anchor_pos)
        heatmap_rect.topleft += Vec2(full_rect.topleft)

        if ret_heatmap_rect:
            return heatmap_rect
        return full_rect

    def logic(self):
        self.past_tensor = self.tensor.detach().clone()
        return super().logic()

    def draw(self, gfx: "GFX"):
        matrix = self.tensor
        assert matrix.shape == self.shape

        """Draw a matrix or vector as a heatmap"""
        if matrix.dim() == 1:
            matrix = matrix.unsqueeze(1)

        if self.title:
            gfx.text(self.title, midtop=self.rect.midtop)

        # Use a symmetric range for the colors
        range_ = max(abs(matrix.min()), abs(matrix.max())).item()
        if range_ == 0:
            range_ = 1

        if matrix.grad is not None:
            grad = matrix.grad
        else:
            grad = matrix.detach() - self.past_tensor
        grad_range = max(abs(grad.min()), abs(grad.max())).item()
        if grad_range == 0:
            grad_range = 1


        heatmap_rect = self.geometry(ret_heatmap_rect=True, **self.anchor)

        for r in range(matrix.shape[0]):
            for c in range(matrix.shape[1]):
                v = matrix[r, c].item()
                pos = Vec2(c, r) * self.neuron_size + heatmap_rect.topleft
                rect = Rect(pos, (self.neuron_size, self.neuron_size))
                gfx.box(rect, self.color(v, range_))
                if grad_range is not None:
                    g = grad[r, c].item()
                    gfx.rect(*rect, self.color(g, grad_range), 3)
                gfx.text(
                    self.fmt_text(v, gfx),
                    15 / gfx.screen_scale,  # For constant screen size
                    "white",
                    center=rect.center,
                )

        gfx.rect(*heatmap_rect, "pink", 1)

        return super().draw(gfx)
        
    def fmt_text(self, v: float, gfx) -> str:
        screen_size = self.neuron_size * gfx.screen_scale

        if screen_size < 15:
            return ""
        f = f"{v:.{int(screen_size // 30)}f}"
        # If only zeros, don't show them
        if all(c == "0" for c in f if c.isdigit()):
            return ""
        return f

    def color(self, v: float, range_: float) -> Tuple[int, int, int]:
        zero_color = (20, 30, 40)
        if v < 0:
            return mix(RED, zero_color, chrange(v, (-range_, 0), (0, 1)))
        else:
            return mix((zero_color), GREEN, chrange(v, (0, range_), (0, 1)))



# For reloading
try:
    t.alive = False
except NameError:
    pass

def watcher():
    head = model.blocks[0].attention.heads[0]
    q = head.queries
    k = head.keys
    return q @ k.T

# t = pyapp.state.add(TensorView(
#     watcher,
#     title="Q 0",
#     topleft=(0, -200),
#     neuron_size=20,
# ))

def draw(gfx: GFX):
    transformer_outline(gfx, model, pos=(0, 0))

Traceback (most recent call last):
  File "/home/diego/ai/hand-transformer/viz/main.py", line 200, in draw
    self.globals.get('draw', lambda gfx: None)(gfx)
  File "/tmp/ipykernel_75028/3021211618.py", line 2, in draw
    r = transformer_outline(gfx, model)
  File "/home/diego/ai/hand-transformer/viz/utils.py", line 50, in transformer_outline
    r = transformer_block_outline(gfx, block, pos=(pos.x, r.bottom + 40))
  File "/home/diego/ai/hand-transformer/viz/utils.py", line 82, in transformer_block_outline
    r2 = mlp_outline(gfx, block.mlp, pos=r.bottomleft)
  File "/home/diego/ai/hand-transformer/viz/utils.py", line 114, in mlp_outline
    r = draw_matrix(gfx,
  File "/home/diego/ai/hand-transformer/viz/utils.py", line 191, in draw_matrix
    rect = Rect(heatmap_rect.x + c * size, heatmap_rect.y + r * size,
NameError: name 'Rect' is not defined


In [53]:
model.embedding.weight.grad

tensor([[-5.7539e-04,  6.0611e-04,  1.8771e-04, -8.9267e-04,  1.7244e-04,
          3.5200e-04, -3.5142e-04,  1.0021e-04, -1.1393e-04,  0.0000e+00,
         -1.9094e-04,  9.3221e-04],
        [ 5.1911e-04, -5.4670e-04, -1.4138e-04,  5.2188e-04, -1.1979e-03,
         -3.1211e-04,  3.1158e-04, -8.9031e-05,  1.0118e-04,  0.0000e+00,
          1.3933e-04, -3.8464e-04]])

In [ ]:

a = (
    "000 0",
    "100 0",
    "101 0",
    "001 1",

    "010 0",
    "011 1",
    "110 1",
    "111 1",
)

b = (
    "000 0",
    "100 0",
    "010 0",
    "110 1",

    "101 0",
    "001 1",
    "011 1",
    "111 1",
)